In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from itertools import islice
import yaml
from hydra.experimental import compose, initialize_config_module
import hydra
import torch
from tqdm import tqdm
import json
import faiss
import logging
from collections import defaultdict, Counter
from dataclasses import dataclass
from typing import Optional, List, Dict, Any, Tuple
from pprint import pprint
from pathlib import Path

from bela.utils.analysis_utils import convert_jsonl_data_to_samples
from bela.transforms.spm_transform import SPMTransform
from bela.evaluation.model_eval import ModelEval, load_file
from bela.utils.prediction_utils import get_predictions_using_windows
from bela.utils.analysis_utils import Entity, Sample, convert_jsonl_data_and_predictions_to_samples

logger = logging.getLogger(__name__)
!cat /etc/hostname  # Double check that we are on a gpu node

a100-st-p4d24xlarge-68


# Load model

In [8]:
%%prun -s tottime -l 10

# e2e model with isotropic embeddings
checkpoint_path = '/checkpoints/movb/bela/2023-01-13-023711/0/lightning_logs/version_4144/checkpoints/last.ckpt'  # Not working: Unexpected key(s) in state_dict: "saliency_encoder.mlp.0.weight", "saliency_encoder.mlp.0.bias", "saliency_encoder.mlp.3.weight", "saliency_encoder.mlp.3.bias", "saliency_encoder.mlp.6.weight", "saliency_encoder.mlp.6.bias". 
checkpoint_path = '/checkpoints/movb/bela/2022-11-27-225013/0/lightning_logs/version_286287/checkpoints/last_15000.ckpt'  # Works but 0 F1

# E2E checkpoint with new embeddings
# https://fb.quip.com/QVUxA4UcAZ7k#temp:C:OcG977f71fab43d42379521a0dff
# Works but give 0 F1 on tackbp (mention detection is okish, but entity disambiguation is random)
checkpoint_path = '/checkpoints/movb/bela/2023-01-18-220105/0/lightning_logs/version_4820/checkpoints/last.ckpt'  
# Overfit on one sample from /fsx/louismartin/bela/data/debug_mention_detection/mel/train.1st.1_sample.txt
checkpoint_path = '/data/home/louismartin/dev/BELA/multirun/2023-02-14/08-45-13/0/lightning_logs/version_0/checkpoints/checkpoint_best.ckpt'
checkpoint_path = "/data/home/louismartin/dev/BELA/multirun/2023-02-15/15-59-33/0/lightning_logs/version_0/checkpoints/checkpoint_best.ckpt"
checkpoint_path = "/data/home/louismartin/dev/BELA/multirun/2023-02-15/16-16-51/0/lightning_logs/version_0/checkpoints/checkpoint_best-v1.ckpt"
# Overfit on 100 samples (.948 f1
checkpoint_path = "/data/home/louismartin/dev/BELA/multirun/2023-02-15/23-28-57/0/lightning_logs/version_0/checkpoints/checkpoint_best.ckpt"
checkpoint_path = "/checkpoints/movb/bela/2023-02-15-200343/0/lightning_logs/version_127953/checkpoints/checkpoint_4.ckpt"
# New fixed checkpoint
checkpoint_path = "/checkpoints/movb/bela/2023-02-15-200343/0/lightning_logs/version_127953/checkpoints/checkpoint_best-v2.ckpt"
# Finetuned on TACKBP
#checkpoint_path = "/data/home/louismartin/dev/BELA/multirun/2023-02-17/00-08-45/0/lightning_logs/version_0/checkpoints/checkpoint_best.ckpt"
checkpoint_path = "/data/home/louismartin/dev/BELA/multirun/2023-02-17/00-08-45/0/lightning_logs/version_0/checkpoints/checkpoint_best-v2.ckpt"
# Finetuned on TACKBP (shuffled data)
checkpoint_path = "/checkpoints/louismartin/bela/2023-02-17-193248/lightning_logs/version_0/checkpoints/checkpoint_best-v1.ckpt"
## Finetuned on pseudo-labeled data / silver data
#checkpoint_path = "/checkpoints/movb/bela/2023-02-17-172049/lightning_logs/version_0/checkpoints/checkpoint_2.ckpt"
# Finetuned on aida
checkpoint_path = "/checkpoints/movb/bela/aida/lightning_logs/version_0/checkpoints/checkpoint_1.ckpt"
model_eval = ModelEval(checkpoint_path, config_name="joint_el_mel_new_index")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


         34358726 function calls (34319027 primitive calls) in 55.779 seconds

   Ordered by: internal time
   List reduced from 1807 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3   15.804    5.268   15.804    5.268 serialization.py:994(load_tensor)
        1   13.157   13.157   13.157   13.157 {built-in method faiss._swigfaiss.GpuIndexFlat_add}
        1   12.731   12.731   14.152   14.152 joint_el_datamodule.py:21(__init__)
        1    2.868    2.868   55.780   55.780 <string>:2(<module>)
        1    1.954    1.954   52.912   52.912 model_eval.py:31(__init__)
      302    1.849    0.006    1.849    0.006 {method 'uniform_' of 'torch._C._TensorBase' objects}
        4    1.847    0.462    1.847    0.462 {method 'normal_' of 'torch._C._TensorBase' objects}
 16470920    1.372    0.000    1.372    0.000 {method 'strip' of 'str' objects}
      396    1.028    0.003    1.028    0.003 {method '_set_from_file' of 'torch._C.

# End-to-end Eval

In [12]:

model_eval.task.md_threshold = 0
model_eval.task.el_threshold = 0
#model_eval.task.md_threshold = 0.2
#model_eval.task.el_threshold = 0.4
#model_eval.task.md_threshold = 0.1
#model_eval.task.el_threshold = 0.1
print(f"{model_eval.checkpoint_path=}")
print(f"{model_eval.task.md_threshold=}")
print(f"{model_eval.task.el_threshold=}")
datasets = [
    #"/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_all_languages.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_spa.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_cmn.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_eng.jsonl",
    #"/fsx/louismartin/bela/data/debug_mention_detection/mel/eval.1st.1_sample.txt",  # Overfit on one sample
    #"/fsx/louismartin/bela/data/debug_mention_detection/100_samples/eval.txt",  # Overfit on 100 samples
    #'/fsx/movb/data/matcha/mewsli-9/ta.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/ar.jsonl',
    ##'/fsx/movb/data/matcha/mewsli-9/en.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/fa.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/sr.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/tr.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/de.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/es.jsonl',
    #'/fsx/movb/data/matcha/mewsli-9/ja.jsonl',
]
for test_data_path in datasets:
    print(f"Processing {test_data_path}")
    test_data = load_file(test_data_path)
    #print("Loading only 100 samples for debugging")
    #test_data = test_data[:100]
    
    predictions = get_predictions_using_windows(model_eval, test_data, window_length=256)
    #predictions = model_eval.get_predictions(test_data)
    (f1, precision, recall), (f1_boe, precision_boe, recall_boe) = ModelEval.compute_scores(test_data, predictions)
    #model_results = ModelResults(test_data, predictions)
    #(f1, precision, recall), (f1_boe, precision_boe, recall_boe) = model_results.compute_scores()
    
    print(f"F1 = {f1:.4f}, precision = {precision:.4f}, recall = {recall:.4f}")
    #print(f"F1 boe = {f1_boe:.4f}, precision = {precision_boe:.4f}, recall = {recall_boe:.4f}")
    samples = convert_jsonl_data_and_predictions_to_samples(jsonl_data=test_data, predictions=predictions, md_threshold=model_eval.task.md_threshold, el_threshold=model_eval.task.el_threshold)
    for sample in islice(samples, 0):
        if len(sample.false_positives) + len(sample.false_negatives) == 0:
            continue
        print("-" * 80)
        print(f"{sample.text[500:]=}")
        print(f"***************** {len(sample.true_positives)} True Positives *****************")
        pprint(sample.true_positives[:5])
        print(f"***************** {len(sample.false_positives)} False Positives *****************")
        pprint(sample.false_positives[:5])
        print(f"***************** {len(sample.false_negatives)} False Negatives *****************")
        pprint(sample.false_negatives[:5])

model_eval.checkpoint_path='/checkpoints/movb/bela/aida/lightning_logs/version_0/checkpoints/checkpoint_1.ckpt'
model_eval.task.md_threshold=0
model_eval.task.el_threshold=0
Processing /fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_spa.jsonl


167it [00:00, 19367.60it/s]
 50%|█████████████████████▌                     | 1/2 [01:04<01:04, 64.61s/it]


AssertionError: ﻿E != El

## Grid search for thresholds

In [ ]:
import numpy as np
from itertools import product
results = {}
for md_threshold, el_threshold in tqdm(list(product(np.arange(0, 0.6, 0.2), repeat=2))):
    model_eval.task.md_threshold = md_threshold
    model_eval.task.el_threshold = el_threshold
    print(f"{model_eval.checkpoint_path=}")
    print(f"{model_eval.task.md_threshold=}")
    print(f"{model_eval.task.el_threshold=}")
    datasets = [
        "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_all_languages.jsonl",
    ]
    for test_data_path in datasets:
        print(f"Processing {test_data_path}")
        test_data = load_file(test_data_path)
        #print("Loading only 100 samples for debugging")
        #test_data = test_data[:100]
        predictions = get_predictions_using_windows(model_eval, test_data, window_length=1024)
        (f1, precision, recall), (f1_boe, precision_boe, recall_boe) = ModelEval.compute_scores(test_data, predictions)
        print(f"F1 = {f1:.4f}, precision = {precision:.4f}, recall = {recall:.4f}")
        print(f"F1 boe = {f1_boe:.4f}, precision = {precision_boe:.4f}, recall = {recall_boe:.4f}")
        results[(md_threshold, el_threshold)] = (f1, precision, recall), (f1_boe, precision_boe, recall_boe)

# Check how many of tackbp entities are in our index

In [7]:
paths = [
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_spa.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_cmn.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_eng.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_spa.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_cmn.jsonl",
    "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_eng.jsonl",
]
print(f"Our entity index size is {len(model_eval.ent_idx)}")
for path in paths:
    print(f"Processing {path}")
    jsonl_data = load_file(path)
    samples = convert_jsonl_data_to_samples(jsonl_data)
    print(f"Loaded {len(samples)} samples")
    counter = Counter()
    for sample in samples:
        counter.update([entity.entity_id for entity in sample.ground_truth_entities])
    unique_entities = counter.keys()
    total_mentions = sum(counter.values())
    entities_not_in_index = [entity_id for entity_id in counter.keys() if entity_id not in model_eval.ent_idx]
    print(f"Total mentions: {total_mentions}")
    print(f"Unique entities: {len(unique_entities)}")
    print(f"Unique entities not in index: {len(entities_not_in_index)}")

Our entity index size is 16470856
Processing /fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_spa.jsonl


167it [00:00, 20000.82it/s]

Loaded 167 samples


Total mentions: 4046
Unique entities: 616
Unique entities not in index: 0
Processing /fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/dev_bela_format_cmn.jsonl


166it [00:00, 462.46it/s]


...

147it [00:00, 2554.48it/s]

Loaded 147 samples


Total mentions: 10520
Unique entities: 800
Unique entities not in index: 0
Processing /fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format_eng.jsonl


168it [00:00, 5683.89it/s]


Loaded 168 samples
Total mentions: 9027
Unique entities: 1055
Unique entities not in index: 0


In [6]:
4046 + 8476

12522

In [5]:
167+166

333

# Disambiguation Eval

In [ ]:

def shift_shift(text):
    for idx,ch in enumerate(text):
        if not ch.isalpha():
            return idx

def convert_data_for_disambiguation(data, lang):
    # convert examples to 1 entity per example and shift if needed
    if lang=='ar':
        MAX_LENGTH = 600
        MAX_OFFSET = 400
    elif lang == 'ja':
        MAX_LENGTH = 350
        MAX_OFFSET = 250
    else:
        MAX_LENGTH = 800
        MAX_OFFSET = 600
    new_examples = []
    for example in tqdm(data):
        original_text = example['original_text']
        for _, _, ent, _, offset, length in example['gt_entities']:
            shift = 0
            if len(original_text) > MAX_LENGTH and offset > MAX_OFFSET:
                shift = (offset - MAX_OFFSET)
                shift += shift_shift(original_text[shift:])
            new_example = {
                'original_text': original_text[shift:],
                'gt_entities': [[0,0,ent,_,offset-shift,length]],
            }
            new_examples.append(new_example)
    return new_examples


def metrics_disambiguation(test_data, predictions):
    support = 0
    support_only_predicted = 0
    correct = 0
    incorrect_pos = 0

    for example_idx, (example, prediction) in tqdm(enumerate(zip(test_data, predictions))):
#         targets = {
#             (offset,length):ent_id
#             for _,_,ent_id,_,offset,length in example['gt_entities']
#         }
#         prediction = {
#             (offset,length):ent_id
#             for offset,length,ent_id in zip(prediction['offsets'], prediction['lengths'], prediction['entities'])
#         }

#         support += len(targets)
#         support_only_predicted += len(prediction)
        
#         correct += sum(1 for pos,ent_id in prediction.items() if (pos in targets and targets[pos] == ent_id))
#         incorrect_pos += sum(1 for pos,_ in prediction.items() if pos not in targets)
        if len(prediction['entities']) == 0:
            continue
        target = example['gt_entities'][0][2]
        prediction = prediction['entities'][0]
        correct += (target == prediction)
        support += 1

    accuracy = correct/support
    # accuracy_only_predicted = correct/support_only_predicted

    return accuracy, support #, accuracy_only_predicted, support_only_predicted

In [ ]:
datasets = [
    '/fsx/movb/data/matcha/mewsli-9/ta.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/ar.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/en.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/fa.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/sr.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/tr.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/de.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/es.jsonl',
    '/fsx/movb/data/matcha/mewsli-9/ja.jsonl',
]
for test_data_path in datasets:
    print(f"Processing {test_data_path}")
    lang = test_data_path[-8:-6]
    test_data = load_file(test_data_path)
    test_data = convert_data_for_disambiguation(test_data[:10000], lang)
    predictions = model_eval.get_disambiguation_predictions(test_data)
    accuracy, support = metrics_disambiguation(test_data, predictions)
    print(f"Accuracty {accuracy}, support {support}")

In [18]:
from bela.transforms.spm_transform import *
len(convert_text_to_spm_tokens_with_char_offsets(line["original_text"], model_eval.transform.processor))

25

In [24]:
from pathlib import Path

def yield_jsonl_lines(path):
    with open(path) as f:
        for line in f:
            yield json.loads(line)


path = Path("/fsx/movb/data/matcha/mel/train.1st.txt")
train_path = Path("/fsx/louismartin/bela/data/debug_mention_detection/100_samples/train.txt")
i = 0
lines = []
for line in tqdm(yield_jsonl_lines(path)):
    if not "is a" in line['original_text']:
        continue
    #if len(line["original_text"]) > 256 * 4:
    #    continue
    if len(convert_text_to_spm_tokens_with_char_offsets(line["original_text"], model_eval.transform.processor)) > 256 * 0.9:
        continue
    lines.append(line)
    i += 1
    if i >= 100:
        break
# Write lines 1000 times in a file
with open(train_path, "w") as f:
    for _ in range(1000):
        for line in lines:
            f.write(json.dumps(line) + "\n")
eval_path = Path("/fsx/louismartin/bela/data/debug_mention_detection/100_samples/eval.txt")
with open(eval_path, "w") as f:
    for line in lines:
        f.write(json.dumps(line) + "\n")
test_path = Path("/fsx/louismartin/bela/data/debug_mention_detection/100_samples/test.txt")
with open(test_path, "w") as f:
    for line in lines:
        f.write(json.dumps(line) + "\n")

1270it [00:00, 17120.65it/s]


## Eyeball Samples

With text directly as input

In [13]:
from bela.evaluation.model_eval import Sample
text = "Her name is Taylor Swift. New York City is a city in the United States."
text = "My dog is a Shiba Inu. Taylor Swift is a singer."
text = "Her name is Taylor Swift."
text = "He is the Dalai Lama. He is a Buddhist monk."
text = "   Martina Steuk (née Kämpfert; born 11 November 1959) is a German former track and field athlete who represented East Germany. She competed in the 800 metres and occasionally the 400 metres.  Her first success came at the 1977 European Athletics Junior Championships, where she won the 800 m title. At twenty years old, reached the final at the 1980 Moscow Olympics and placed fourth in a lifetime best time of 1:56.21 behind a Soviet trio of Nadiya Olizarenko, Olga Mineyeva and Tatyana Providokhina that broke the world record. A successful 1981 season followed, which included a win at the 1981 European Cup, and at the 1981 IAAF World Cup she won an 800 m silver behind Lyudmila Veselkova and a gold with the East German 4 × 400 metres relay team. She ranked second in the world that year for the 800 m behind Vesselkova with a time of 1:57.16. That success continued into 1982 with a silver medal at the 1982 European Athletics Indoor Championships, finishing second only to Romania's Doina Melinte. She was top of the world indoor rankings that season with 1:59.24 minutes.  She was a three-time national champion, winning the 800 m at the East German Athletics Championships in 1981 before taking a national indoor and outdoor double in 1982.  She holds one of the fastest times for the 1000 metres at 2:30.85 minutes. This was the second fastest ever when it was set in 1980, behind Soviet runner Tatyana Providokhina, though it remained the fastest recorded electronically recorded time for ten years. It remains the best mark by an under-23 athlete and she also holds the under-23 best for the 600 metres event with 1:24.56 minutes.  Steuk was born in Berlin and was a member of her local club Berliner TSC. She married East German hammer thrower Roland Steuk in 1991. The two subsequently divorced. She was awarded the Patriotic Order of Merit for her athletic feats in 1982. "
prediction = model_eval.process_batch([text])[0]
predicted_entities = [Entity(offset=offset, length=length, text=text, entity_id=entity_id, md_score=md_score, el_score=el_score) for offset, length, entity_id, md_score, el_score in zip(prediction["offsets"], prediction["lengths"], prediction["entities"], prediction["md_scores"], prediction["el_scores"])]
Sample(text=text, predicted_entities=predicted_entities).print()

self.text[:max_display_length]="   Martina Steuk (née Kämpfert; born 11 November 1959) is a German former track and field athlete who represented East Germany. She competed in the 800 metres and occasionally the 400 metres.  Her first success came at the 1977 European Athletics Junior Championships, where she won the 800 m title. At twenty years old, reached the final at the 1980 Moscow Olympics and placed fourth in a lifetime best time of 1:56.21 behind a Soviet trio of Nadiya Olizarenko, Olga Mineyeva and Tatyana Providokhina that broke the world record. A successful 1981 season followed, which included a win at the 1981 European Cup, and at the 1981 IAAF World Cup she won an 800 m silver behind Lyudmila Veselkova and a gold with the East German 4 × 400 metres relay team. She ranked second in the world that year for the 800 m behind Vesselkova with a time of 1:57.16. That success continued into 1982 with a silver medal at the 1982 European Athletics Indoor Championships, finishing se

From file with windows

In [150]:

test_data_path = "/fsx/louismartin/bela/retrieved_from_aws_backup/ndecao/TACKBP2015/train_bela_format.jsonl"
test_data_path = "/fsx/louismartin/bela/data/debug_mention_detection/mel/eval.1st.1_sample.txt"  # Overfit on one sample
print(f"Processing {test_data_path}")
test_data = load_file(test_data_path)
#sample = test_data[200]
sample = test_data[0]
prediction = get_predictions_using_windows(model_eval, [sample], window_length=1024)[0]
text = sample["original_text"]
max_length = 1024

ground_truth_entities = [Entity(offset=offset, length=length, text=text, entity_id=entity_id) for _, _, entity_id, _, offset, length in sample["gt_entities"]]
predicted_entities = [Entity(offset=offset, length=length, text=text, entity_id=entity_id, md_score=md_score, el_score=el_score) for offset, length, entity_id, md_score, el_score in zip(prediction["offsets"], prediction["lengths"], prediction["entities"], prediction["md_scores"], prediction["el_scores"])]
print_sample(text, ground_truth_entities, predicted_entities, max_display_length=100000)

Processing /fsx/louismartin/bela/data/debug_mention_detection/mel/eval.1st.1_sample.txt


78it [00:00, 22171.03it/s]
100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.42it/s]

['▁track', '▁and', '▁field']

['▁East', '▁Germany', '.']

['▁metres']

['▁metres', '.']

['▁1977', '▁European', '▁Athletic', 's', '▁Junior', '▁Championship', 's', ',']

['▁1980', '▁Moscow', '▁Olympic', 's']

['diya', '▁O', 'lizar', 'enko', ',']

['diya', '▁O', 'lizar', 'enko', ',', '▁Olga', '▁Mine', 'yev', 'a']

['▁Olga', '▁Mine', 'yev', 'a']

['yana', '▁Pro', 'vid', 'ok', 'hina']

['▁1981', '▁season']

['▁1981', '▁European', '▁Cup', ',']

['▁1981', '▁I', 'A', 'AF', '▁World', '▁Cup']

['ud', 'mila', '▁Vesel', 'kova']

['▁400', '▁metres', '▁rela', 'y']

['▁1982', '▁European', '▁Athletic', 's', '▁In', 'door', '▁Championship', 's', ',']

['ina', '▁Mel', 'inte', '.']

text[:max_display_length]="   Martina Steuk (née Kämpfert; born 11 November 1959) is a German former track and field athlete who represented East Germany. She competed in the 800 metres and occasionally the 400 metres.  Her first success came at the 1977 European Athletics Junior Championships, where she won the 800 m title. 

# Debug mention detection bug

In [31]:
print(ground_truth_entities[0].offset, ground_truth_entities[0].length)
print(predicted_entities[0].offset, predicted_entities[0].length)

74 15
74 23


In [8]:
# Set low thresholds
model_eval.task.md_threshold = 0.1
model_eval.task.el_threshold = 0.1
text = "Taylor Swift is a singer."
prediction = model_eval.process_batch([text])[0]
predicted_entities = [PredictedEntity(offset, length, text, entity_id, md_score, el_score) for offset, length, entity_id, md_score, el_score in zip(prediction["offsets"], prediction["lengths"], prediction["entities"], prediction["md_scores"], prediction["el_scores"])]
print_sample(text, [], predicted_entities)

text[:max_display_length]='Taylor Swift is a singer.'
***************** Ground truth entities *****************
len(ground_truth_entities)=0
***************** Predicted entities *****************
len(predicted_entities)=1
mention="Taylor Swift" -> entity_id=Q26876 (md_score=0.15, el_score=0.02)


## SPM Encode and Decode

In [25]:
text = "400 metres.  Her"
print(model_eval.transform.processor.encode(text))
print(model_eval.transform.processor.decode([4, 1839]))

[4081, 106383, 4, 1839]
. Her


In [111]:


sp_offset = 19
sp_length = 4
#text = "   Martina Steuk (née Kämpfert; born 11 November 1959) is a German former track and field athlete who represented East Germany. She competed in the 800 metres and occasionally the 400 metres.  Her first success came at the 1977 European Athletics Junior Championships, where she won the 800 m title. At twenty years old, reached the final at the 1980 Moscow Olympics and placed fourth in a lifetime best time of 1:56.21 behind a Soviet trio of Nadiya Olizarenko, Olga Mineyeva and Tatyana Providokhina that broke the world record. A successful 1981 season followed, which included a win at the 1981 European Cup, and at the 1981 IAAF World Cup she won an 800 m silver behind Lyudmila Veselkova and a gold with the East German 4 × 400 metres relay team. She ranked second in the world that year for the 800 m behind Vesselkova with a time of 1:57.16. That success continued into 1982 with a silver medal at the 1982 European Athletics Indoor Championships, finishing second only to Romania's Doina Melinte."
#token_ids = model_eval.transform.processor.encode(text)
#mention_token_ids = token_ids[sp_offset:sp_offset+sp_length]
#[model_eval.transform.processor.decode([token_id]) for token_id in mention_token_ids]
convert_sp_to_char_offsets(text, sp_offset, sp_length, model_eval.transform.processor)


['track', 'and', 'field']
Martina Steuk (née Kämpfert; born 11 November 1959) is a German former


(70, 15)

In [ ]:
def test_convert_sp_to_char_offsets(spm_processor):
    text = "   Martina Steuk (née Kämpfert; born 11 November 1959) is a German former track and field athlete who represented East Germany.   "
    sp_offset = 19
    sp_length = 4
    sp_offset -= 1  # sp_offsets include cls_token, while boundaries doesn't
    sp_length -= 1  # TODO: it is not clear why we need to subtract 1
    char_offset, char_length = convert_sp_to_char_offsets(text, sp_offset, sp_length, spm_processor)
    assert text[char_offset: char_offset + char_length] == "track and field", f"Expected 'track and field', got '{text[char_offset: char_offset + char_length]}'"


spm_processor = model_eval.transform.processor
spm_processor.EncodeAsPieces(text)
test_sentencepiece_to_char_conversion(spm_processor)

# Inference time

In [6]:
def batch_samples(samples, batch_size):
    # Yield batches of samples
    for i in range(0, len(samples), batch_size):
        yield samples[i : i + batch_size]


texts = [sample["original_text"] for sample in load_file("/fsx/movb/data/matcha/mewsli-9/en.jsonl")]
batch_size = 1024
print(f"Processing {len(texts)} texts, {batch_size=}, {model_eval.transform.max_seq_len=}")
%time _ = [model_eval.process_batch(batch_texts) for batch_texts in tqdm(batch_samples(texts, batch_size), desc="Inference")]

12679it [00:00, 70671.85it/s]


Processing 12679 texts, batch_size=1024, model_eval.transform.max_seq_len=256


13it [04:00, 18.53s/it]

CPU times: user 11min 50s, sys: 35.3 s, total: 12min 25s
Wall time: 4min


# Draft

In [ ]:

class Sample:
    text: str
    ground_truth_entities: List[Entity]
    predicted_entities: List[Entity]

    def __init__(self, text, ground_truth_entities, predicted_entities):
        self.text = text
        self.ground_truth_entities = ground_truth_entities
        self.predicted_entities = predicted_entities
        # Compute scores
        self.true_positives = [predicted_entity for predicted_entity in self.predicted_entities if predicted_entity in self.ground_truth_entities]
        self.false_positives = [predicted_entity for predicted_entity in self.predicted_entities if predicted_entity not in self.ground_truth_entities]
        self.false_negatives = [ground_truth_entity for ground_truth_entity in self.ground_truth_entities if ground_truth_entity not in self.predicted_entities]
        # Bag of entities
        gold_entity_ids = set([ground_truth_entity.entity_id for ground_truth_entity in self.ground_truth_entities])
        predicted_entity_ids = set([predicted_entity.entity_id for predicted_entity in self.predicted_entities])
        self.true_positives_boe = [predicted_entity_id for predicted_entity_id in predicted_entity_ids if predicted_entity_id in gold_entity_ids]
        self.false_positives_boe = [predicted_entity_id for predicted_entity_id in predicted_entity_ids if predicted_entity_id not in gold_entity_ids]
        self.false_negatives_boe = [ground_truth_entity_id for ground_truth_entity_id in gold_entity_ids if ground_truth_entity_id not in predicted_entity_ids]


    def print(self, max_display_length=1000):
        print(f"{self.text[:max_display_length]=}")
        print("***************** Ground truth entities *****************")
        print(f"{len(self.ground_truth_entities)=}")
        for ground_truth_entity in self.ground_truth_entities:
            if ground_truth_entity.offset + ground_truth_entity.length > max_display_length:
                continue
            print(ground_truth_entity)
        print("***************** Predicted entities *****************")
        print(f"{len(self.predicted_entities)=}")
        for predicted_entity in self.predicted_entities:
            if predicted_entity.offset + predicted_entity.length > max_display_length:
                continue
            print(predicted_entity)



class ModelResults:
    def __init__(self, data, predictions, md_threshold=0.2, el_threshold=0.05, verbose=False):
        self.data = data
        self.predictions = predictions
        self.md_threshold = md_threshold
        self.el_threshold = el_threshold
        self.verbose = verbose
        self.samples = []
        self._compute_scores()

    def _compute_scores(self):
        self.samples = []
        for ground_truth_sample, predicted_sample in zip(self.data, self.predictions):
            ground_truth_entities = [
                GroundTruthEntity(
                    offset=offset,
                    length=length,
                    text=ground_truth_sample['original_text'],
                    entity_id=ent_id,
                )
                for offset, length, ent_id, _, _, _ in ground_truth_sample['gt_entities']
            ]
            predicted_entities = [
                PredictedEntity(
                    offset=offset,
                    length=length,
                    text=ground_truth_sample['original_text'],
                    entity_id=ent_id,
                    md_score=md_score,
                    el_score=el_score,
                )
                for offset, length, ent_id, md_score, el_score in zip(
                    predicted_sample['offsets'],
                    predicted_sample['lengths'],
                    predicted_sample['entities'],
                    predicted_sample['md_scores'],
                    predicted_sample['el_scores'],
                )
                if (el_score > self.el_threshold and md_score > self.md_threshold)
            ]
            sample = Sample(
                text=ground_truth_sample['original_text'],
                ground_truth_entities=ground_truth_entities,
                predicted_entities=predicted_entities,
            )
            self.samples.append(sample)
        
        def safe_division(numerator, denominator):
            return numerator / denominator if denominator > 0 else 0.0

        self.precision = safe_division(
            sum([len(sample.true_positives) for sample in self.samples]),
            sum([len(sample.predicted_entities) for sample in self.samples]),
        )
        self.recall = safe_division(
            sum([len(sample.true_positives) for sample in self.samples]),
            # TODO: Check that we can't predict the same entity twice
            sum([len(sample.ground_truth_entities) for sample in self.samples]),
        )
        self.f1 = safe_division(2 * self.precision * self.recall, self.precision + self.recall)
        self.precision_boe = safe_division(
            sum([len(sample.true_positives_boe) for sample in self.samples]),
            sum([len(sample.predicted_entities) for sample in self.samples]),
        )
        self.recall_boe = safe_division(
            sum([len(sample.true_positives_boe) for sample in self.samples]),
            sum([len(sample.ground_truth_entities) for sample in self.samples]),
        )
        self.f1_boe = safe_division(2 * self.precision_boe * self.recall_boe, self.precision_boe + self.recall_boe)
        #return (f1, precision, recall), (f1_boe, precision_boe, recall_boe)
